# Bayesian Optimization for LightGBM  
- prashantkikani kenel (https://www.kaggle.com/prashantkikani/rstudio-lgb-single-model-lb1-6607) 
- Bayesian Optimization kernel (https://www.kaggle.com/sz8416/simple-bayesian-optimization-for-lightgbm)
- stratified kernel (https://www.kaggle.com/youhanlee/stratified-sampling-for-regression-lb-1-6595)

prashantkikani (https://www.kaggle.com/prashantkikani/rstudio-lgb-single-model-lb1-6607) feature engineering code using.

In [ ]:
'''
Adding some new features..
Combining categorical features.
'''

import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import time
from datetime import datetime
import gc
import psutil
from sklearn.preprocessing import LabelEncoder

PATH="../input/"
NUM_ROUNDS = 20000
VERBOSE_EVAL = 500
STOP_ROUNDS = 100
N_SPLITS = 10

 #the columns that will be parsed to extract the fields from the jsons
cols_to_parse = ['device', 'geoNetwork', 'totals', 'trafficSource']

def read_parse_dataframe(file_name):
    #full path for the data file
    path = PATH + file_name
    #read the data file, convert the columns in the list of columns to parse using json loader,
    #convert the `fullVisitorId` field as a string
    data_df = pd.read_csv(path, 
        converters={column: json.loads for column in cols_to_parse}, 
        dtype={'fullVisitorId': 'str'})
    #parse the json-type columns
    for col in cols_to_parse:
        #each column became a dataset, with the columns the fields of the Json type object
        json_col_df = json_normalize(data_df[col])
        json_col_df.columns = [f"{col}_{sub_col}" for sub_col in json_col_df.columns]
        #we drop the object column processed and we add the columns created from the json fields
        data_df = data_df.drop(col, axis=1).merge(json_col_df, right_index=True, left_index=True)
    return data_df
    
def process_date_time(data_df):
    print("process date time ...")
    data_df['date'] = data_df['date'].astype(str)
    data_df["date"] = data_df["date"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:])
    data_df["date"] = pd.to_datetime(data_df["date"])   
    data_df["year"] = data_df['date'].dt.year
    data_df["month"] = data_df['date'].dt.month
    data_df["day"] = data_df['date'].dt.day
    data_df["weekday"] = data_df['date'].dt.weekday
    data_df['weekofyear'] = data_df['date'].dt.weekofyear
    data_df['month_unique_user_count'] = data_df.groupby('month')['fullVisitorId'].transform('nunique')
    data_df['day_unique_user_count'] = data_df.groupby('day')['fullVisitorId'].transform('nunique')
    data_df['weekday_unique_user_count'] = data_df.groupby('weekday')['fullVisitorId'].transform('nunique')

    return data_df

def process_format(data_df):
    print("process format ...")
    for col in ['visitNumber', 'totals_hits', 'totals_pageviews']:
        data_df[col] = data_df[col].astype(float)
    data_df['trafficSource_adwordsClickInfo.isVideoAd'].fillna(True, inplace=True)
    data_df['trafficSource_isTrueDirect'].fillna(False, inplace=True)
    return data_df
    
def process_device(data_df):
    print("process device ...")
    data_df['browser_category'] = data_df['device_browser'] + '_' + data_df['device_deviceCategory']
    data_df['browser_os'] = data_df['device_browser'] + '_' + data_df['device_operatingSystem']
    return data_df

def process_totals(data_df):
    print("process totals ...")
    data_df['visitNumber'] = np.log1p(data_df['visitNumber'])
    data_df['totals_hits'] = np.log1p(data_df['totals_hits'])
    data_df['totals_pageviews'] = np.log1p(data_df['totals_pageviews'].fillna(0))
    data_df['mean_hits_per_day'] = data_df.groupby(['day'])['totals_hits'].transform('mean')
    data_df['sum_hits_per_day'] = data_df.groupby(['day'])['totals_hits'].transform('sum')
    data_df['max_hits_per_day'] = data_df.groupby(['day'])['totals_hits'].transform('max')
    data_df['min_hits_per_day'] = data_df.groupby(['day'])['totals_hits'].transform('min')
    data_df['var_hits_per_day'] = data_df.groupby(['day'])['totals_hits'].transform('var')
    data_df['mean_pageviews_per_day'] = data_df.groupby(['day'])['totals_pageviews'].transform('mean')
    data_df['sum_pageviews_per_day'] = data_df.groupby(['day'])['totals_pageviews'].transform('sum')
    data_df['max_pageviews_per_day'] = data_df.groupby(['day'])['totals_pageviews'].transform('max')
    data_df['min_pageviews_per_day'] = data_df.groupby(['day'])['totals_pageviews'].transform('min')
    data_df['max_pageviews_WoY'] = data_df.groupby(['weekofyear'])['totals_pageviews'].transform('max')
    data_df['min_pageviews_WoY'] = data_df.groupby(['weekofyear'])['totals_pageviews'].transform('min')
    data_df['hit_vs_views'] = data_df['totals_pageviews'] / data_df['totals_hits']
    data_df['hit_vs_views_r'] = data_df['totals_hits'] / data_df['totals_pageviews']
    return data_df

def process_geo_network(data_df):
    print("process geo network ...")
    data_df['sum_pageviews_per_network_domain'] = data_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('sum')
    data_df['count_pageviews_per_network_domain'] = data_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('count')
    data_df['mean_pageviews_per_network_domain'] = data_df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('mean')
    data_df['sum_hits_per_network_domain'] = data_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('sum')
    data_df['count_hits_per_network_domain'] = data_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('count')
    data_df['mean_hits_per_network_domain'] = data_df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('mean')
    return data_df

def process_traffic_source(data_df):
    print("process traffic source ...")
    data_df['source_country'] = data_df['trafficSource_source'] + '_' + data_df['geoNetwork_country']
    data_df['campaign_medium'] = data_df['trafficSource_campaign'] + '_' + data_df['trafficSource_medium']
    data_df['medium_hits_mean'] = data_df.groupby(['trafficSource_medium'])['totals_hits'].transform('mean')
    data_df['medium_hits_max'] = data_df.groupby(['trafficSource_medium'])['totals_hits'].transform('max')
    data_df['medium_hits_min'] = data_df.groupby(['trafficSource_medium'])['totals_hits'].transform('min')
    data_df['medium_hits_sum'] = data_df.groupby(['trafficSource_medium'])['totals_hits'].transform('sum')
    return data_df
    
def custom(data):
    print('custom..')
    for i in ['geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country','geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region','geoNetwork_subContinent']:
        for j in ['device_browser','device_deviceCategory', 'device_operatingSystem', 'trafficSource_source']:
            data[i + "_" + j] = data[i] + "_" + data[j]

    return data

#Feature processing
## Load data
print('reading train')
train_df = read_parse_dataframe('train.csv')
trn_len = train_df.shape[0]
train_df = process_date_time(train_df)
print('reading test')
test_df = read_parse_dataframe('test.csv')
test_df = process_date_time(test_df)

## Drop columns
cols_to_drop = [col for col in train_df.columns if train_df[col].nunique(dropna=False) == 1]
train_df.drop(cols_to_drop, axis=1, inplace=True)
test_df.drop([col for col in cols_to_drop if col in test_df.columns], axis=1, inplace=True)

###only one not null value
train_df.drop(['trafficSource_campaignCode'], axis=1, inplace=True)

###converting columns format
train_df['totals_transactionRevenue'] = train_df['totals_transactionRevenue'].astype(float)
train_df['totals_transactionRevenue'] = train_df['totals_transactionRevenue'].fillna(0)
train_df['totals_transactionRevenue'] = np.log1p(train_df['totals_transactionRevenue'])


## Features engineering
train_df = process_format(train_df)
train_df = process_device(train_df)
train_df = process_totals(train_df)
train_df = process_geo_network(train_df)
train_df = process_traffic_source(train_df)
train_df = custom(train_df)

test_df = process_format(test_df)
test_df = process_device(test_df)
test_df = process_totals(test_df)
test_df = process_geo_network(test_df)
test_df = process_traffic_source(test_df)
test_df = custom(test_df)

## Categorical columns
print("process categorical columns ...")
num_cols = ['month_unique_user_count', 'day_unique_user_count', 'weekday_unique_user_count',
            'visitNumber', 'totals_hits', 'totals_pageviews', 
            'mean_hits_per_day', 'sum_hits_per_day', 'min_hits_per_day', 'max_hits_per_day', 'var_hits_per_day',
            'mean_pageviews_per_day', 'sum_pageviews_per_day', 'min_pageviews_per_day', 'max_pageviews_per_day',
            'sum_pageviews_per_network_domain', 'count_pageviews_per_network_domain', 'mean_pageviews_per_network_domain',
            'sum_hits_per_network_domain', 'count_hits_per_network_domain', 'mean_hits_per_network_domain',
            'medium_hits_mean','medium_hits_min','medium_hits_max','medium_hits_sum','hit_vs_views','hit_vs_views_r',
            'max_pageviews_WoY','min_pageviews_WoY']
            
not_used_cols = ["visitNumber", "date", "fullVisitorId", "sessionId", 
        "visitId", "visitStartTime", 'totals_transactionRevenue', 'trafficSource_referralPath']
cat_cols = [col for col in train_df.columns if col not in num_cols and col not in not_used_cols]

for col in cat_cols:
    #print(col)
    lbl = LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

print('FINAL train shape : ', train_df.shape, ' test shape : ', test_df.shape)
#print(train_df.columns)
train_df = train_df.sort_values('date')
X = train_df.drop(not_used_cols, axis=1)
y = train_df['totals_transactionRevenue']
X_test = test_df.drop([col for col in not_used_cols if col in test_df.columns], axis=1)

## Step 1: parameters to be tuned

In [ ]:
import warnings
import time
warnings.filterwarnings("ignore")
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
train_data = lgb.Dataset(data=X, label=y, categorical_feature = list(X.columns),free_raw_data=False)
    
cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False,shuffle=False)

In [ ]:
-1.0 * np.array(cv_result['rmse-mean'])

In [ ]:
(-1.0 * np.array(cv_result['rmse-mean'])).max()

In [ ]:
def lgb_eval(num_leaves, feature_fraction, max_depth , min_split_gain, min_child_weight):
    params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['max_depth'] = int(round(max_depth))
    params['num_leaves'] = int(round(num_leaves))
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False)
    return (-1.0 * np.array(cv_result['rmse-mean'])).max()

In [ ]:
lgbBO = BayesianOptimization(lgb_eval, {'feature_fraction': (0.1, 0.9),
                                            'max_depth': (5, 9),
                                            'num_leaves' : (200,300),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)

## Step 2: minimize rmse

In [ ]:
lgbBO.maximize(init_points=5, n_iter=5,acq='ei')

## Step 3: minimize rmse

In [ ]:
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=0, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, categorical_feature = list(X.columns),free_raw_data=False)
    # parameters

    def lgb_eval(num_leaves, feature_fraction, max_depth , min_split_gain, min_child_weight):
        params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['num_leaves'] = int(round(num_leaves))
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, verbose_eval =200,stratified=False)
        return (-1.0 * np.array(cv_result['rmse-mean'])).max()
    
        # range 
    lgbBO = BayesianOptimization(lgb_eval, {'feature_fraction': (0.1, 0.9),
                                            'max_depth': (5, 9),
                                            'num_leaves' : (200,300),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
        # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round,acq='ei')

        # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")

        # return best parameters
    return lgbBO.res['max']['max_params']

opt_params = bayes_parameter_opt_lgb(X, y, init_round=10, opt_round=10, n_folds=5, random_seed=0, n_estimators=1000, learning_rate=0.01)

In [ ]:
print(opt_params)

## STEP 3: Check Result1
prashantkikani kenel (https://www.kaggle.com/prashantkikani/rstudio-lgb-single-model-lb1-6607) 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import model_selection, preprocessing, metrics
import matplotlib.pyplot as plt
import seaborn as sns

lgb_params = {'num_leaves': 201,
             'min_data_in_leaf': 30, 
             'objective':'regression',
             'max_depth': 9,
             'learning_rate': 0.01,
             "min_child_samples": 50,
             "boosting": "rf",
             "feature_fraction": 0.7941654378759639,
             "min_split_gain" : 0.025580954346285312,
             "min_child_weight" : 49.94864050157583,
             "bagging_freq": 1,
             "bagging_fraction": 0.8,
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 1,
             "verbosity": -1
             }
             
lgb_params1 = {"objective" : "regression", "metric" : "rmse", 
               "feature_fraction": 0.7941654378759639,
               "min_split_gain" : 0.025580954346285312,
               "min_child_weight" : 49.94864050157583,
               "max_depth": 9, "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,
               "num_leaves" : 201, "learning_rate" : 0.01, "subsample" : 0.8, 
               "colsample_bytree" : 0.8, "verbosity": -1}

run_lgb = True
print('LGB : ', run_lgb)
# modeling
#--------------------------------------------------------------------------
if run_lgb:
    import lightgbm as lgb
    def kfold_lgb_xgb():
        folds = KFold(n_splits=5, shuffle=True, random_state=7)
        
        oof_lgb = np.zeros(len(train_df))
        predictions_lgb = np.zeros(len(test_df))

        features_lgb = list(X.columns)
        feature_importance_df_lgb = pd.DataFrame()

        for fold_, (trn_idx, val_idx) in enumerate(folds.split(X)):
            trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
            val_data = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])
            
            print("LGB " + str(fold_) + "-" * 50)
            num_round = 20000
            clf = lgb.train(lgb_params1, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 100)
            oof_lgb[val_idx] = clf.predict(X.iloc[val_idx], num_iteration=clf.best_iteration)
            
            fold_importance_df_lgb = pd.DataFrame()
            fold_importance_df_lgb["feature"] = features_lgb
            fold_importance_df_lgb["importance"] = clf.feature_importance()
            fold_importance_df_lgb["fold"] = fold_ + 1
            feature_importance_df_lgb = pd.concat([feature_importance_df_lgb, fold_importance_df_lgb], axis=0)
            predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
        
        #lgb.plot_importance(clf, max_num_features=30)    
        cols = feature_importance_df_lgb[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:50].index
        best_features_lgb = feature_importance_df_lgb.loc[feature_importance_df_lgb.feature.isin(cols)]
        plt.figure(figsize=(14,10))
        sns.barplot(x="importance", y="feature", data=best_features_lgb.sort_values(by="importance", ascending=False))
        plt.title('LightGBM Features (avg over folds)')
        plt.tight_layout()
        plt.savefig('lgbm_importances.png')
        x = []
        for i in oof_lgb:
            if i < 0:
                x.append(0.0)
            else:
                x.append(i)
        cv_lgb = mean_squared_error(x, y)**0.5
        cv_lgb = str(cv_lgb)
        cv_lgb = cv_lgb[:10]
        
        pd.DataFrame({'preds': x}).to_csv('lgb_oof_' + cv_lgb + '.csv', index = False)
        
        print("CV_LGB : ", cv_lgb)
        return cv_lgb, predictions_lgb
        
    cv_lgb, lgb_ans = kfold_lgb_xgb()
    x = []
    for i in lgb_ans:
        if i < 0:
            x.append(0.0)
        else:
            x.append(i)
    np.save('lgb_ans.npy', x)
    submission = test_df[['fullVisitorId']].copy()
    submission.loc[:, 'PredictedLogRevenue'] = x
    submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"].apply(lambda x : 0.0 if x < 0 else x)
    submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"].fillna(0.0)
    grouped_test = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
    grouped_test.to_csv('lgb_' + cv_lgb + '.csv',index=False)

# STEP 3 : Result check2  
kernel : Stratified sampling for regression LB: 1.6595  
https://www.kaggle.com/youhanlee/stratified-sampling-for-regression-lb-1-6595  

In [ ]:
def categorize_target(x):
    if x < 2:
        return 0
    elif x < 4:
        return 1
    elif x < 6:
        return 2
    elif x < 8:
        return 3
    elif x < 10:
        return 4
    elif x < 12:
        return 5
    elif x < 14:
        return 6
    elif x < 16:
        return 7
    elif x < 18:
        return 8
    elif x < 20:
        return 9
    elif x < 22:
        return 10
    else:
        return 11
    
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
y_categorized = y.apply(categorize_target)

In [ ]:
import lightgbm as lgb
def kfold_lgb_xgb():
    FOLDs = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

    oof_lgb = np.zeros(len(train_df))
    predictions_lgb = np.zeros(len(test_df))

    features_lgb = list(X.columns)
    feature_importance_df_lgb = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(FOLDs.split(X, y_categorized)):
        trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
        val_data = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])

        print("LGB " + str(fold_) + "-" * 50)
        num_round = 20000
        clf = lgb.train(lgb_params1, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 100)
        oof_lgb[val_idx] = clf.predict(X.iloc[val_idx], num_iteration=clf.best_iteration)

        fold_importance_df_lgb = pd.DataFrame()
        fold_importance_df_lgb["feature"] = features_lgb
        fold_importance_df_lgb["importance"] = clf.feature_importance()
        fold_importance_df_lgb["fold"] = fold_ + 1
        feature_importance_df_lgb = pd.concat([feature_importance_df_lgb, fold_importance_df_lgb], axis=0)
        predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / FOLDs.n_splits

    #lgb.plot_importance(clf, max_num_features=30)    
    cols = feature_importance_df_lgb[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:50].index
    best_features_lgb = feature_importance_df_lgb.loc[feature_importance_df_lgb.feature.isin(cols)]
    plt.figure(figsize=(14,10))
    sns.barplot(x="importance", y="feature", data=best_features_lgb.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')
    x = []
    for i in oof_lgb:
        if i < 0:
            x.append(0.0)
        else:
            x.append(i)
    cv_lgb = mean_squared_error(x, y)**0.5
    cv_lgb = str(cv_lgb)
    cv_lgb = cv_lgb[:10]

    pd.DataFrame({'preds': x}).to_csv('lgb_oof_' + cv_lgb + '.csv', index = False)

    print("CV_LGB : ", cv_lgb)
    return cv_lgb, predictions_lgb

cv_lgb, lgb_ans = kfold_lgb_xgb()
x = []
for i in lgb_ans:
    if i < 0:
        x.append(0.0)
    else:
        x.append(i)
np.save('lgb_ans.npy', x)
submission = test_df[['fullVisitorId']].copy()
submission.loc[:, 'PredictedLogRevenue'] = x
submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"].apply(lambda x : 0.0 if x < 0 else x)
submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"].fillna(0.0)
grouped_test = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
grouped_test.to_csv('Stratified_lgb_' + cv_lgb + '.csv',index=False)

## Result

- https://www.kaggle.com/prashantkikani/rstudio-lgb-single-model-lb1-6607 is LB: 1.6607 **==>**

- https://www.kaggle.com/youhanlee/stratified-sampling-for-regression-lb-1-6595 is LB : 1.6595. **==>**